In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
#the usual suspects
import pickle
import pandas as pd
import warnings

from pair_trading.preproc import alph_settings as alph, alph_api_wrapper as wrap, threading, helper
import pair_trading.model as model

print(dir())
warnings.filterwarnings('ignore')

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '__vsc_ipynb_file__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'alph', 'baseline_df', 'best_params', 'data', 'datetime', 'exit', 'f', 'features_df', 'get_ipython', 'helper', 'log_reg', 'model', 'pair', 'pd', 'pickle', 'quit', 'random_forest', 're', 'signal_df', 'svm_df', 'svm_pred', 'threading', 'warnings', 'wrap']


In [3]:
pair = ["FRO", "ELP"]
features_df = model.get_features(pair)
display(features_df)

,VWAP,SMA(5),SMA(10),12dayEWM,RSI,MACD,MOM,MFI,spread
time,,,,,,,,,
2021-04-14 14:00:00,1.375654,1.378488,1.377690,1.368880,1.029191,-0.045507,-0.007252,-2.190030,1.681399
2021-04-14 15:00:00,1.374333,1.373638,1.376446,1.364915,1.023536,-0.045591,0.060702,-1.520747,1.687958
2021-04-14 16:00:00,1.372977,1.366281,1.378651,1.369549,1.125999,-0.038760,0.147987,20.564874,1.767896
2021-04-14 17:00:00,1.372811,1.352773,1.373749,1.355536,1.012141,-0.046334,-0.106520,32.303330,1.600327
2021-04-15 09:00:00,1.372955,1.389060,1.386539,1.383672,1.159748,-0.007922,0.071108,26.242060,1.843639
...,...,...,...,...,...,...,...,...,...
2023-03-31 13:00:00,1.774769,2.447931,2.446051,2.447709,0.744290,-0.118742,-0.010000,-6.402698,9.685000
2023-03-31 14:00:00,1.774846,2.453594,2.448981,2.456580,0.936113,-0.102094,0.170000,-2.062330,9.805000
2023-03-31 15:00:00,1.774912,2.454776,2.449985,2.456311,0.816358,-0.095467,0.090000,-10.095378,9.710000


In [4]:
signal_df = model.get_labels(features_df, 24, 0.02)
display(signal_df)


            Count of Trades Identified:
            Buy signals: 1917
            Sell signals: 1765
            Hold signals: 407
            


,VWAP,SMA(5),SMA(10),12dayEWM,RSI,MACD,MOM,MFI,spread,forward_return,output
time,,,,,,,,,,,
2021-04-14 14:00:00,1.375654,1.378488,1.377690,1.368880,1.029191,-0.045507,-0.007252,-2.190030,1.681399,NaN,0
2021-04-14 15:00:00,1.374333,1.373638,1.376446,1.364915,1.023536,-0.045591,0.060702,-1.520747,1.687958,NaN,0
2021-04-14 16:00:00,1.372977,1.366281,1.378651,1.369549,1.125999,-0.038760,0.147987,20.564874,1.767896,NaN,0
2021-04-14 17:00:00,1.372811,1.352773,1.373749,1.355536,1.012141,-0.046334,-0.106520,32.303330,1.600327,NaN,0
2021-04-15 09:00:00,1.372955,1.389060,1.386539,1.383672,1.159748,-0.007922,0.071108,26.242060,1.843639,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 13:00:00,1.774769,2.447931,2.446051,2.447709,0.744290,-0.118742,-0.010000,-6.402698,9.685000,-0.055756,-1
2023-03-31 14:00:00,1.774846,2.453594,2.448981,2.456580,0.936113,-0.102094,0.170000,-2.062330,9.805000,-0.043855,-1
2023-03-31 15:00:00,1.774912,2.454776,2.449985,2.456311,0.816358,-0.095467,0.090000,-10.095378,9.710000,-0.047374,-1


In [5]:
baseline_df = model.run_baseline(features_df, 24, 0.05)

              precision    recall  f1-score   support

          -1       0.67      0.01      0.01       347
           0       0.11      0.48      0.17        83
           1       0.78      0.88      0.82       383

    accuracy                           0.47       813
   macro avg       0.52      0.46      0.34       813
weighted avg       0.66      0.47      0.41       813



In [31]:
svm_pred, best_params = model.run_svm(features_df)

<function cross_val_score at 0x0000022AFF7CDFC0>
GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=SVC(class_weight='balanced',
                           decision_function_shape='ovo'),
             param_grid=[{'C': [1, 10, 100], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100], 'kernel': ['linear']},
                         {'C': [1, 10, 100], 'degree': [2, 3, 4],
                          'gamma': ['auto'], 'kernel': ['poly']}],
             scoring='f1_weighted')

        Best Parameters: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
        Best Cross-Validation Score: 1.0
        


              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       282
           0       1.00      1.00      1.00       199
           1       1.00      1.00      1.00       308

    accuracy                           1.00       789
   m

In [15]:
log_reg = model.run_logistic_regression(features_df)

Best: 0.798887 using {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'lbfgs'}
0.537210 (0.022754) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'sag'}
0.530853 (0.026239) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
0.601575 (0.022148) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'newton-cg'}
0.601984 (0.022127) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
0.567653 (0.025943) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'liblinear'}
0.601575 (0.022020) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'sag'}
0.600037 (0.022106) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}
0.798887 (0.021649) with: {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
0.768033 (0.021157) with: {'C': 0.001, 'penalty': 'none', 'solver': 'lbfgs'}
0.0000

In [17]:
random_forest = model.run_random_forest(features_df)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
              precision    recall  f1-score   support

          -1       0.88      0.65      0.75       347
           0       0.12      0.11      0.11        83
           1       0.75      0.94      0.84       383

    accuracy                           0.73       813
   macro avg       0.58      0.57      0.57       813
weighted avg       0.74      0.73      0.72       813



In [21]:
#Individual model best parameters
with open('svm_params.pkl', 'rb') as f:
    svm_params = pickle.load(f)
print(svm_params)
with open('logreg_params.pkl', 'rb') as f:
    logreg_params = pickle.load(f)
print(logreg_params)
with open('rf_params.pkl', 'rb') as f:
    rf_params = pickle.load(f)
print(rf_params)

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
{'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
{'bootstrap': True, 'max_depth': 80, 'max_features': 9, 'min_samples_leaf': 4, 'min_samples_split': 12, 'n_estimators': 100}


In [32]:
svm_model, logreg_model, rf_model = model.make_models(svm_params, logreg_params, rf_params)
stacking_pred = model.run_stacking(features_df, svm_model, logreg_model, rf_model)
max_voting_pred = model.run_max_voting(features_df, svm_model, logreg_model, rf_model)

              precision    recall  f1-score   support

          -1       0.89      0.70      0.79       347
           0       0.00      0.00      0.00        83
           1       0.68      0.96      0.80       383

    accuracy                           0.75       813
   macro avg       0.53      0.55      0.53       813
weighted avg       0.70      0.75      0.71       813

0.7527675276752768
              precision    recall  f1-score   support

          -1       0.86      0.72      0.79       347
           0       0.15      0.07      0.10        83
           1       0.74      0.93      0.83       383

    accuracy                           0.76       813
   macro avg       0.58      0.58      0.57       813
weighted avg       0.73      0.76      0.74       813

0.7564575645756457
